# Load the USGS LACP MODFLOW 6 Model

Culling, D.P., Paulinski, S.R., and Rush, M.J., 2022, MODFLOW-6 model to update and extend the Los Angeles Coastal Plain Groundwater Model: U.S. Geological Survey data release, https://doi.org/10.5066/P9TJD4IE.

In [ ]:
import os
import contextily as cx
import folium
import numpy as np
import flopy

## Helpful Functions and Variable Definitions

In [ ]:
from contextlib import contextmanager
@contextmanager
def change_dir(destination):
    current_dir = os.getcwd()
    os.chdir(destination)
    try:
        yield
    finally:
        os.chdir(current_dir)


def get_gridprops():
    nlay = 12
    nrow = 256
    ncol = 312
    dx = 659.96646693 / 3.2808
    dy = dx
    xll = 355046.48  
    yll = 3777032.06 - ncol * dy
    rotation = 0.

    delr = np.empty((ncol), dtype=float)
    delr.fill(dx)
    delc = np.empty((nrow), dtype=float)
    delc.fill(dy)

    gridprops = {
        "nlay": nlay,
        "nrow": nrow,
        "ncol": ncol,
        "delr": delr,
        "delc": delc,
        "xoff": xll,
        "yoff": yll,
        "angrot": rotation,
        "crs": "EPSG:26911",
    }

    return gridprops

def get_modelgrid():
    return flopy.discretization.StructuredGrid(
        **get_gridprops()
    )

esri_tile = folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    overlay=False,
    control=True,
)

## Load and Interrogate the Model

In [ ]:
ws = "../lacp"
with change_dir(ws):
    sim = flopy.mf6.MFSimulation.load()

In [ ]:
sim.model_names

In [ ]:
gwf = sim.get_model(sim.model_names[0])

In [ ]:
gwf.get_package_list()

In [ ]:
# accessing data
npf = gwf.npf
hk = npf.k.get_data()
hk.min(), hk.max()

## Simplest Plotting

In [ ]:
# plot by array
# gwf.dis.top.plot()

In [ ]:
# plot by package
# gwf.dis.plot()

## Customizable Plotting

In [ ]:
# pxs = flopy.plot.PlotCrossSection(gwf, line={"row": 100})
# pxs.plot_grid()
# pxs.plot_array(npf.k.array)

In [ ]:
# pmv = flopy.plot.PlotMapView(gwf, layer=0)
# pmv.plot_bc("GHB")
# pmv.plot_bc("DRN")
# # pmv.plot_array(npf.k.array, cmap="viridis", vmin=hk.min(), vmax=hk.max())
# pmv.plot_inactive(alpha=0.5)

## Better Plots

In [ ]:
modelgrid = get_modelgrid()
gdf = modelgrid.geo_dataframe
gdf

In [ ]:
# gdf.plot()

In [ ]:
gdf["idomain0"] = gwf.dis.idomain.array[0].flatten()
gdf

In [ ]:
gdf_idomain = gdf.dissolve(by="idomain0", aggfunc="sum", as_index=False)
gdf_idomain = gdf_idomain.drop(0)
gdf_idomain

In [ ]:
gdf_idomain.explore(tiles=esri_tile, cmap="jet", legend=True, figsize=(10, 10))

In [ ]:
ax = gdf.plot(figsize=(10, 10), alpha=0.5, facecolor="none", edgecolor="black", lw=0.5)
source_url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
source_url = None
cx.add_basemap(ax, crs=gdf.crs, source=source_url)

In [ ]:
ax = gdf.plot(figsize=(10, 10), alpha=0.5, facecolor="none", edgecolor="black", lw=0.1)
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax)
pmv.plot_inactive(ibound=gwf.dis.idomain.array[1], alpha=0.25)
source_url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
# source_url = None
cx.add_basemap(ax, crs=gdf.crs, source=source_url)